In [55]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
import torchvision.transforms as transforms
import numpy as np
import time
torch.cuda.device_count()
#使えるGPU数

2

In [44]:
transform = transforms.Compose([
    transforms.ToTensor()
    ])
trainset = torchvision.datasets.CIFAR10(root='../dataset/',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=128,shuffle=True,num_workers=2)

testset = torchvision.datasets.CIFAR10(root='../dataset/',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=128,shuffle=False,num_workers=2)
#データセットの読みこみ

Files already downloaded and verified
Files already downloaded and verified


In [45]:
trainset[0][0].shape

torch.Size([3, 32, 32])

In [48]:

model = nn.Sequential(
    nn.Conv2d(3,32,kernel_size=3,stride=2,padding=1),
    nn.ReLU(),
    nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
    nn.ReLU(),
    nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
    nn.ReLU(),
    nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
    nn.ReLU(),
    nn.Conv2d(256,512,kernel_size=3,stride=2,padding=1),
    nn.ReLU(),
    nn.Conv2d(512,1024,kernel_size=3,stride=1,padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(output_size=(1,1)),
    nn.Flatten(),
    nn.Linear(1024,128),
    nn.Dropout(0.2),
    nn.ReLU(),
    nn.Linear(128,10),
    nn.Dropout(0.2),
    nn.Softmax(dim=1)
)
print(model(torch.zeros((1,3,32,32))))
#モデルの定義
lossfunc = nn.CrossEntropyLoss()
#分類タスクなのでCrossEntropyLoss
softmax = nn.Softmax()
device = "cuda"
model = model.to(device)
if True:
    model = nn.DataParallel(model)
#torch.nn.DataParallelにモデルを投げることでデータ並列化
optim = opt.Adam(model.parameters())
#学習はAdam Optimizerを用いる

tensor([[0.1025, 0.0995, 0.1086, 0.0964, 0.0997, 0.0996, 0.1025, 0.1025, 0.0935,
         0.0952]], grad_fn=<SoftmaxBackward>)


In [49]:
epoch = 10
start = time.time()
#学習をepoch数だけ回す
for ep in range(epoch):
    for i, (img, label) in enumerate(trainloader):
        img = img.to(device)
        label = label.to(device)
        optim.zero_grad()
        output = model(img)
        loss = lossfunc(output, label)
        loss.backward()
        #print(torch.sum(torch.argmax(output, axis=1)==label)*100/label.shape[0])
        optim.step()
    acc = 0
    num = 0
    #1epoch回すごとにテストデータで分類率を確かめる
    with torch.no_grad():
        for i, (img, label) in enumerate(testloader):
            img = img.to(device)
            label = label.to(device)
            output = model(img)
            acc += int(torch.sum(torch.argmax(output, axis=1)==label).detach().cpu())
            #最も出力が大きいものを推測ラベルとしてあっているものを数える
            num += label.shape[0]
    print("epoch%d"%ep,loss)
    print(acc/num*100,"%")
#全epochが終わった時点の経過時間を計測
end = time.time()
print("time:", end-start)

epoch0 tensor(2.1739, device='cuda:0', grad_fn=<NllLossBackward>)
25.7 %
epoch1 tensor(2.1341, device='cuda:0', grad_fn=<NllLossBackward>)
30.0 %
epoch2 tensor(2.1653, device='cuda:0', grad_fn=<NllLossBackward>)
30.73 %
epoch3 tensor(2.0460, device='cuda:0', grad_fn=<NllLossBackward>)
34.339999999999996 %
epoch4 tensor(2.1202, device='cuda:0', grad_fn=<NllLossBackward>)
37.11 %
epoch5 tensor(2.1823, device='cuda:0', grad_fn=<NllLossBackward>)
37.44 %
epoch6 tensor(2.0869, device='cuda:0', grad_fn=<NllLossBackward>)
38.32 %
epoch7 tensor(2.1366, device='cuda:0', grad_fn=<NllLossBackward>)
36.08 %
epoch8 tensor(2.0803, device='cuda:0', grad_fn=<NllLossBackward>)
37.669999999999995 %
epoch9 tensor(2.0098, device='cuda:0', grad_fn=<NllLossBackward>)
42.78 %
time: 92.3054940700531


batch_size = 128のとき

並列時 time: 92.3054940700531

非並列時 time: 75.03011393547058

batch_size = 256のとき

並列時 time: 62.5077702999115

非並列時 time: 70.36796069145203

batch_size = 512のとき

並列時 time: 50.71963453292847
    
非並列時time: 65.97711610794067

バッチサイズが小さいと並列化しない方が早かったがバッチサイズを上げるごとに並列時の方が早くなった。
モデルをさらに大きくすると違いが大きくなると考える。

### 参考文献

PytorchでMulti-GPUを試す https://qiita.com/arutema47/items/2b92f94c734b0a11609d
    
PyTorch Tutorials OPTIONAL: DATA PARALLELISM https://pytorch.org/tutorials/beginner/blitz/data_parallel_tutorial.html